# Chapter 3: Processing Pipelines

- https://course.spacy.io/chapter2

# 1)- Inspecting the pipeline

In [2]:
import spacy

# Load the en_core_web_sm model
nlp = spacy.load("en_core_web_sm")

In [3]:
# Print the names of the pipeline components
print(nlp.pipe_names)

['tagger', 'parser', 'ner']


In [4]:
# Print the full pipeline of (name, component) tuples
print(nlp.pipeline)

[('tagger', <spacy.pipeline.pipes.Tagger object at 0x11dfb5050>), ('parser', <spacy.pipeline.pipes.DependencyParser object at 0x116dda3d0>), ('ner', <spacy.pipeline.pipes.EntityRecognizer object at 0x116dda360>)]


# 2)-custom Pipeline Components